In [1]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.array as da
import dask.bag as db

# Introducing DASK
## Python-native parallel computing framework
## Programming 3 Lecture 6

# DASK: python-native parallel computing
- We've seen several different forms of parallelization;
    1. from-scratch (python multiprocessing)
    2. external UNIX-style (SLURM, GNU parallel)
    3. Data-center style (SPARK)
- Each had its own strengths, weaknesses and ugliness
- Now it's time to look at the latest form; native, scientific-python based DASK

# DASK
- Dask is composed of two parts:

    1. Dynamic task scheduling optimized for computation. This is similar to Airflow, Luigi, Celery, or Make, but optimized for interactive computational workloads.
    2. “Big Data” collections like parallel arrays, dataframes, and lists that extend common interfaces like NumPy, Pandas, or Python iterators to larger-than-memory or distributed environments. These parallel collections run on top of dynamic task schedulers.

# Dask Advantages
- Familiar: Provides parallelized NumPy array and Pandas DataFrame objects
- Flexible: Provides a task scheduling interface for more custom workloads and integration with other projects.
- Native: Enables distributed computing in pure Python with access to the PyData stack.
- Fast: Operates with low overhead, low latency, and minimal serialization necessary for fast numerical algorithms
- Scales up: Runs resiliently on clusters with 1000s of cores
- Scales down: Trivial to set up and run on a laptop in a single process
- Responsive: Designed with interactive computing in mind, it provides rapid feedback and diagnostics to aid humans

![Overview](dask-overview.svg)

In [2]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.array as da
import dask.bag as db

# Using Dask for simple Python parallelization
- If you're just looking to parallize à la Multiprocessing
- Any python "iterable" can be used this way!
    - I.e. needs to implement the "__next__" and "__iter__" functions!
- We can make a distributed version of ordinary Python objects using "Bags"
- You program mostly in the functional paradigm

In [3]:
b = db.from_sequence([1, 2, 3, 4, 5, 6, 2, 1], npartitions=2)

- Dask, like Spark, is lazy; nothing happens until you force a result
- E.g. use `.compute()` to force the data to be rendered

In [4]:
b.compute()

[1, 2, 3, 4, 5, 6, 2, 1]

In [5]:
b

dask.bag<from_sequence, npartitions=2>

- You mostly program with Python's _functional_ features:

In [6]:
b.filter(lambda x: x % 2)

dask.bag<filter-lambda, npartitions=2>

In [7]:
b.filter(lambda x: x % 2).compute()

[1, 3, 5, 1]

In [8]:
b.distinct()

dask.bag<distinct-aggregate, npartitions=1>

In [9]:
b.distinct().compute()

[1, 2, 3, 4, 5, 6]

In [10]:
c = db.zip(b, b.map(lambda x: x * 10))

In [11]:
c

dask.bag<zip, npartitions=2>

In [12]:
c.compute()

[(1, 10), (2, 20), (3, 30), (4, 40), (5, 50), (6, 60), (2, 20), (1, 10)]

In [13]:
c.dask

HighLevelGraph with 3 layers.
<dask.highlevelgraph.HighLevelGraph object at 0x7fd9704f4640>
 0. from_sequence-cca2a33ba6e12645a0c9bc0fd3fe6c88
 1. lambda-26813ca9e6b0ed3454fb6474c1f75cf4
 2. zip-d1c52c3bdb2fa2ec6206a2a9e5e03fc0

# Numpy parallelization
- Part of where Dask shines is by _overlaying_ the APIs of commons libraries
- E.g. numpy, pandas, scikit-learn, pytorch etc. etc.

![Array](./dask-array.svg)

# Dask follows Numpy API:

- Arithmetic and scalar mathematics: +, *, exp, log, ...
- Reductions along axes: sum(), mean(), std(), sum(axis=0), ...
- Tensor contractions / dot products / matrix multiply: tensordot
- Axis reordering / transpose: transpose
- Slicing: x[:100, 500:100:-2]
- Fancy indexing along single axes with lists or NumPy arrays: x[:, [10, 1, 5]]
- Array protocols like __array__ and __array_ufunc__
- Some linear algebra: svd, qr, solve, solve_triangular, lstsq

In [14]:
data = np.arange(100_000).reshape(200, 500)

In [15]:
data

array([[    0,     1,     2, ...,   497,   498,   499],
       [  500,   501,   502, ...,   997,   998,   999],
       [ 1000,  1001,  1002, ...,  1497,  1498,  1499],
       ...,
       [98500, 98501, 98502, ..., 98997, 98998, 98999],
       [99000, 99001, 99002, ..., 99497, 99498, 99499],
       [99500, 99501, 99502, ..., 99997, 99998, 99999]])

In [16]:
a = da.from_array(data, chunks=(100, 100))

In [17]:
a

dask.array<array, shape=(200, 500), dtype=int64, chunksize=(100, 100), chunktype=numpy.ndarray>

In [18]:
a[:50, 200]

dask.array<getitem, shape=(50,), dtype=int64, chunksize=(50,), chunktype=numpy.ndarray>

In [19]:
a[:50, 0].compute()

array([    0,   500,  1000,  1500,  2000,  2500,  3000,  3500,  4000,
        4500,  5000,  5500,  6000,  6500,  7000,  7500,  8000,  8500,
        9000,  9500, 10000, 10500, 11000, 11500, 12000, 12500, 13000,
       13500, 14000, 14500, 15000, 15500, 16000, 16500, 17000, 17500,
       18000, 18500, 19000, 19500, 20000, 20500, 21000, 21500, 22000,
       22500, 23000, 23500, 24000, 24500])

In [20]:
a.mean()

dask.array<mean_agg-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [21]:
a.mean().compute()

49999.5

In [22]:
a.mean().dask

HighLevelGraph with 5 layers.
<dask.highlevelgraph.HighLevelGraph object at 0x7fd9704fc3d0>
 0. array-9a60a15022e606714732481b91fd2e55
 1. mean_chunk-0d8f2526cf4b25e1e8f1c93422f194ed
 2. mean_combine-partial-2762f86cfca810a54d110bb086349b86
 3. mean_combine-partial-b131f9742c80e76381a262b8e88d9054
 4. mean_agg-aggregate-dc1d99055cbda179f18ca7bdb25e79f6

In [23]:
np.sin(a)

dask.array<sin, shape=(200, 500), dtype=float64, chunksize=(100, 100), chunktype=numpy.ndarray>

In [24]:
np.sin(a).compute()

array([[ 0.        ,  0.84147098,  0.90929743, ...,  0.58781939,
         0.99834363,  0.49099533],
       [-0.46777181, -0.9964717 , -0.60902011, ..., -0.89796748,
        -0.85547315, -0.02646075],
       [ 0.82687954,  0.9199906 ,  0.16726654, ...,  0.99951642,
         0.51387502, -0.4442207 ],
       ...,
       [-0.99720859, -0.47596473,  0.48287891, ..., -0.76284376,
         0.13191447,  0.90539115],
       [ 0.84645538,  0.00929244, -0.83641393, ...,  0.37178568,
        -0.5802765 , -0.99883514],
       [-0.49906936,  0.45953849,  0.99564877, ...,  0.10563876,
         0.89383946,  0.86024828]])

In [25]:
np.sin(a).dask

HighLevelGraph with 2 layers.
<dask.highlevelgraph.HighLevelGraph object at 0x7fd970a1eb20>
 0. array-9a60a15022e606714732481b91fd2e55
 1. sin-395a36ae91b119284c4adf1de925f400

In [26]:
a.T

dask.array<transpose, shape=(500, 200), dtype=int64, chunksize=(100, 100), chunktype=numpy.ndarray>

In [27]:
a.T.compute()

array([[    0,   500,  1000, ..., 98500, 99000, 99500],
       [    1,   501,  1001, ..., 98501, 99001, 99501],
       [    2,   502,  1002, ..., 98502, 99002, 99502],
       ...,
       [  497,   997,  1497, ..., 98997, 99497, 99997],
       [  498,   998,  1498, ..., 98998, 99498, 99998],
       [  499,   999,  1499, ..., 98999, 99499, 99999]])

In [28]:
a.T.dask

HighLevelGraph with 2 layers.
<dask.highlevelgraph.HighLevelGraph object at 0x7fd970a0d550>
 0. array-9a60a15022e606714732481b91fd2e55
 1. transpose-f68a130aaa6522e797154672b0e84760

In [29]:
b = a.max(axis=1)[::-1] + 10

In [30]:
b[:10].compute()

array([100009,  99509,  99009,  98509,  98009,  97509,  97009,  96509,
        96009,  95509])

In [31]:
b[:10].dask

HighLevelGraph with 7 layers.
<dask.highlevelgraph.HighLevelGraph object at 0x7fd970a17df0>
 0. array-9a60a15022e606714732481b91fd2e55
 1. amax-213cb3f5a031c613ebaeca7e2c350af7
 2. amax-partial-c81949c047072f54e849f51de62ec43c
 3. amax-aggregate-ce8cc2b1179559300cf1211e7190f271
 4. getitem-1e81a8142560f1112bc567aa9d1b0028
 5. add-0ec518ef6aeb59b22730b7924913d390
 6. getitem-f553c51dd12551aadb604a23ef187e12

# About numpy...
- Essentially all parallisable numpy functions have been implemented
- Note that some functions are not parallelisable from python (or at all)
- ... if you're already using that, look into Pandas as well!

# Pandas in Dask
Dask DataFrame is used in situations where pandas is commonly needed, usually when pandas fails due to data size or computation speed:

- Manipulating large datasets, even when those datasets don’t fit in memory
- Accelerating long computations by using many cores
- Distributed computing on large datasets with standard pandas operations like groupby, join, and time series - computations

# Don't do Pandas in Dask when:
- If your dataset fits comfortably into RAM on your laptop, then you may be better off just using pandas. There may be simpler ways to improve performance than through parallelism
- If your dataset doesn’t fit neatly into the pandas tabular model, then you might find more use in dask.bag or dask.array
- If you need functions that are not implemented in Dask DataFrame, then you might want to look at dask.delayed which offers more flexibility
- If you need a proper database with all that databases offer you might prefer something like Postgres

# Dask follows Pandas API:
## Trivially parallelizable operations (fast):
- Element-wise operations: df.x + df.y, df * df
- Row-wise selections: df[df.x > 0]
- Loc: df.loc[4.0:10.5]
- Common aggregations: df.x.max(), df.max()
- Is in: df[df.x.isin([1, 2, 3])]
- Date time/string accessors: df.timestamp.month


# Dask follows Pandas API:
## Cleverly parallelizable operations (fast):
- groupby-aggregate (with common aggregations): df.groupby(df.x).y.max(), df.groupby('x').min() (see Aggregate)
- groupby-apply on index: df.groupby(['idx', 'x']).apply(myfunc), where idx is the index level name
- value_counts: df.x.value_counts()
- Drop duplicates: df.x.drop_duplicates()
- Join on index: dd.merge(df1, df2, left_index=True, right_index=True) or dd.merge(df1, df2, on=['idx', 'x']) where idx is the index name for both df1 and df2
- Join with pandas DataFrames: dd.merge(df1, df2, on='id')
- Element-wise operations with different partitions / divisions: df1.x + df2.y
- Date time resampling: df.resample(...)
- Rolling averages: df.rolling(...)
- Pearson’s correlation: df[['col1', 'col2']].corr()

In [32]:
index = pd.date_range("2021-09-01", periods=2400, freq="1H")
df = pd.DataFrame({"a": np.arange(2400), "b": list("abcaddbe" * 300)}, index=index)
ddf = dd.from_pandas(df, npartitions=10)
ddf

,a,b
npartitions=10,,
2021-09-01 00:00:00,int64,object
2021-09-11 00:00:00,...,...
...,...,...
2021-11-30 00:00:00,...,...
2021-12-09 23:00:00,...,...


In [33]:
ddf.divisions

(Timestamp('2021-09-01 00:00:00', freq='H'),
 Timestamp('2021-09-11 00:00:00', freq='H'),
 Timestamp('2021-09-21 00:00:00', freq='H'),
 Timestamp('2021-10-01 00:00:00', freq='H'),
 Timestamp('2021-10-11 00:00:00', freq='H'),
 Timestamp('2021-10-21 00:00:00', freq='H'),
 Timestamp('2021-10-31 00:00:00', freq='H'),
 Timestamp('2021-11-10 00:00:00', freq='H'),
 Timestamp('2021-11-20 00:00:00', freq='H'),
 Timestamp('2021-11-30 00:00:00', freq='H'),
 Timestamp('2021-12-09 23:00:00', freq='H'))

In [34]:
ddf.partitions[1]

,a,b
npartitions=1,,
2021-09-11,int64,object
2021-09-21,...,...


In [35]:
ddf.b

Dask Series Structure:
npartitions=10
2021-09-01 00:00:00    object
2021-09-11 00:00:00       ...
                        ...  
2021-11-30 00:00:00       ...
2021-12-09 23:00:00       ...
Name: b, dtype: object
Dask Name: getitem, 20 tasks

In [36]:
ddf["2021-10-01": "2021-10-09 5:00"]

,a,b
npartitions=1,,
2021-10-01 00:00:00.000000000,int64,object
2021-10-09 05:00:59.999999999,...,...


In [37]:
ddf["2021-10-01": "2021-10-09 5:00"].dask

HighLevelGraph with 2 layers.
<dask.highlevelgraph.HighLevelGraph object at 0x7fd970a17310>
 0. from_pandas-0b850a81e4dfe2d272df4dc718065116
 1. loc-ed0128ca7284d15d6744e99ffb7d076d

In [38]:
ddf.a.mean()

dd.Scalar<series-..., dtype=float64>

In [39]:
ddf.a.mean().compute()

1199.5

In [40]:
ddf.b.unique()

Dask Series Structure:
npartitions=1
    object
       ...
Name: b, dtype: object
Dask Name: unique-agg, 33 tasks

In [41]:
result = ddf["2021-10-01": "2021-10-09 5:00"].a.cumsum() - 100

In [42]:
result

Dask Series Structure:
npartitions=1
2021-10-01 00:00:00.000000000    int64
2021-10-09 05:00:59.999999999      ...
Name: a, dtype: int64
Dask Name: sub, 16 tasks

In [43]:
result.compute()

2021-10-01 00:00:00       620
2021-10-01 01:00:00      1341
2021-10-01 02:00:00      2063
2021-10-01 03:00:00      2786
2021-10-01 04:00:00      3510
                        ...  
2021-10-09 01:00:00    158301
2021-10-09 02:00:00    159215
2021-10-09 03:00:00    160130
2021-10-09 04:00:00    161046
2021-10-09 05:00:00    161963
Freq: H, Name: a, Length: 198, dtype: int64

# The "immediate" Interface
- Runs your code in parallel, immediately
- Take some setting up with clients etc.

In [44]:
from dask.distributed import Client

client = Client()

def inc(x):
   return x + 1

def add(x, y):
   return x + y

a = client.submit(inc, 1)     # work starts immediately
b = client.submit(inc, 2)     # work starts immediately
c = client.submit(add, a, b)  # work starts immediately

c = c.result()

In [45]:
c

5

In [46]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:49318,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:49336,Total threads: 2
Dashboard: http://127.0.0.1:49338/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:49323,


# The delayed Interface
- If you just want to start a bunch of jobs "in the background"
- Only requires some decorators to setup; don't need to start up the "cluster"
- No control over execution; Dask figures out the best "Task Graph"

In [47]:
import dask

@dask.delayed
def inc(x):
   return x + 1

@dask.delayed
def add(x, y):
   return x + y

a = inc(1)       # no work has happened yet
b = inc(2)       # no work has happened yet
c = add(a, b)    # no work has happened yet

c = c.compute()

In [48]:
c

5